In [1]:
import os

In [2]:
%pwd

'c:\\Users\\asus\\OneDrive\\Desktop\\End-To-End-Deep-Learning-Project-With-MLflow-and-DVCM\\research'

In [3]:

os.chdir("../")

In [4]:
%pwd


'c:\\Users\\asus\\OneDrive\\Desktop\\End-To-End-Deep-Learning-Project-With-MLflow-and-DVCM'

In [5]:
import tensorflow as tf

In [6]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [8]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [9]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/kidney-ct-scan-image",
            mlflow_uri="https://dagshub.com/rushikesh092002/End-To-End-Deep-Learning-Project-With-MLflow-and-DVCM.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [10]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse
import dagshub


In [11]:

class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _valid_generator(self):
        datagenerator_kwargs = dict(
            rescale=1. / 255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)

    def evaluation(self):
        print("🔍 Loading model...")
        self.model = self.load_model(self.config.path_of_model)

        print("📦 Preparing validation data generator...")
        self._valid_generator()

        print("✅ Evaluating model...")
        self.score = self.model.evaluate(self.valid_generator)
        print(f"📊 Evaluation Score: Loss = {self.score[0]}, Accuracy = {self.score[1]}")

        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)
        print("💾 Scores saved to scores.json")

    def log_into_mlflow(self):
        # ✅ Initialize DagsHub logging
        dagshub.init(
            repo_owner='rushikesh092002',
            repo_name='End-To-End-Deep-Learning-Project-With-MLflow-and-DVCM',
            mlflow=True
        )

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            print("🚀 Logging parameters and metrics to MLflow (via DagsHub)...")
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics({
                "loss": self.score[0],
                "accuracy": self.score[1]
            })

            print("📦 Logging model to MLflow...")
            if tracking_url_type_store != "file":
                mlflow.keras.log_model(
                    self.model,
                    "model",
                    registered_model_name="VGG16Model"
                )
            else:
                mlflow.keras.log_model(self.model, "model")

            print("✅ MLflow logging complete!")

In [12]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2025-04-10 00:42:01,784: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-04-10 00:42:01,808: INFO: common: yaml file: params.yaml loaded successfully]
[2025-04-10 00:42:01,814: INFO: common: created directory at: artifacts]
🔍 Loading model...
📦 Preparing validation data generator...
Found 139 images belonging to 2 classes.
✅ Evaluating model...
9/9 [==============================] - 31s 3s/step - loss: 8.9935 - accuracy: 0.5036
📊 Evaluation Score: Loss = 8.993518829345703, Accuracy = 0.5035971403121948
[2025-04-10 00:42:34,997: INFO: common: json file saved at: scores.json]
💾 Scores saved to scores.json


❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

c:\Users\asus\anaconda3\envs\kidney\lib\site-packages\rich\live.py:231: UserWarning: install "ipywidgets" for 
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=ee7c26bf-831e-42b8-8963-34efa2c41255&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=1483266b81765a9107c9c4b0649ac85d38e1396a6853f0dbe30bb4aef1c98921


[2025-04-10 00:42:40,156: INFO: _client: HTTP Request: POST https://dagshub.com/login/oauth/middleman "HTTP/1.1 200 OK"]


[2025-04-10 00:42:41,314: INFO: _client: HTTP Request: POST https://dagshub.com/login/oauth/access_token "HTTP/1.1 200 OK"]
[2025-04-10 00:42:42,286: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"]


...

🚀 Logging parameters and metrics to MLflow (via DagsHub)...


2025/04/10 00:42:45 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


📦 Logging model to MLflow...
[2025-04-10 00:42:48,103: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\asus\AppData\Local\Temp\tmp859ii1wl\model\data\model\assets
[2025-04-10 00:42:49,080: INFO: builder_impl: Assets written to: C:\Users\asus\AppData\Local\Temp\tmp859ii1wl\model\data\model\assets]


c:\Users\asus\anaconda3\envs\kidney\lib\site-packages\_distutils_hack\__init__.py:30: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
Successfully registered model 'VGG16Model'.
2025/04/10 00:44:02 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 1
Created version '1' of model 'VGG16Model'.


✅ MLflow logging complete!
